In [2]:
import json
import math
import glob
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from collections import OrderedDict

In [3]:
BSM_SIZE = 101347646 # Already counted
N_FEATURES = 22
root_dir = 'VeReMi-Dataset/'
filenames = glob.glob(root_dir + '**/traceJSON*.json', recursive=True)

In [4]:
# # Count the number of BSM messages in the dataset
# \line_num = 0
# for filepath in tqdm(filenames):
#     f = open(filepath)
#     for line in f:
#         if '"type":3' in line:
#             line_num += 1
    
# line_num

In [5]:
attacker_sender_map = {}

datasets = OrderedDict()

for filepath in filenames:
    path_parts = filepath.split('/')
    filename = path_parts[-1]
    dataset_name = path_parts[1]
    _, vehicle, _, attacker = filename.split('-')[:4]
    attacker_type = int(attacker[1:])
    vehicle_id = int(vehicle)
    attacker_sender_map[f'{dataset_name}:{vehicle_id}'] = attacker_type
    datasets[dataset_name] = 1

for i, k in enumerate(datasets.keys()):
    datasets[k] = i

datasets

OrderedDict([('DelayedMessages_0709', 0),
             ('ConstPos_0709', 1),
             ('RandomSpeedOffset_0709', 2),
             ('RandomSpeed_0709', 3),
             ('DoSRandom_1416', 4),
             ('MixAll_0024', 5),
             ('DoS_0709', 6),
             ('ConstSpeedOffset_1416', 7),
             ('ConstSpeedOffset_0709', 8),
             ('ConstPosOffset_1416', 9),
             ('EventualStop_0709', 10),
             ('DataReplay_1416', 11),
             ('DoSRandom_0709', 12),
             ('ConstSpeed_1416', 13),
             ('EventualStop_1416', 14),
             ('DelayedMessages_1416', 15),
             ('GridSybil_1416', 16),
             ('Disruptive_1416', 17),
             ('DoSRandomSybil_1416', 18),
             ('Disruptive_0709', 19),
             ('DoSDisruptiveSybil_1416', 20),
             ('DoSDisruptive_1416', 21),
             ('DoSDisruptiveSybil_0709', 22),
             ('RandomPos_1416', 23),
             ('DataReplaySybil_1416', 24),
           

In [6]:
veremi = np.memmap('veremi.memmap', dtype='float32', mode='w+', shape=(BSM_SIZE, N_FEATURES))
# filenames = filenames[:1000]

veremi_pos = 0
for filepath in (pbar := tqdm(filenames)):
    f = open(filepath)
    path_parts = filepath.split('/')
    filename = path_parts[-1]
    dataset_name = path_parts[1]
    receiver = int(filename.split('-')[1])
    dataset_id = datasets[dataset_name]
    pbar.set_description("Vehicle ID {}".format(receiver))
    gps = None
    for line in f:
        data = json.loads(line)
        if data['type'] == 2:
            gps = data
        elif data['type'] == 3:
            attack_type = attacker_sender_map.get(f'{dataset_name}:{data["sender"]}', None)
            if attack_type is not None:
                out = (
                    dataset_id,
                    data['messageID'],
                    data['rcvTime'],
                    receiver,
                    data['sender'],
                    attack_type,
                    data['pos'][0],
                    data['pos'][1],
                    data['spd'][0],
                    data['spd'][1],
                    data['acl'][0],
                    data['acl'][1],
                    data['hed'][0],
                    data['hed'][1],
                    gps['pos'][0],
                    gps['pos'][1],
                    gps['spd'][0],
                    gps['spd'][1],
                    gps['acl'][0],
                    gps['acl'][1],
                    gps['hed'][0],
                    gps['hed'][1],
                )
                veremi[veremi_pos] = out
                veremi_pos += 1

  0%|          | 0/134082 [00:00<?, ?it/s]

In [7]:
veremi = np.memmap('veremi.memmap', dtype='float32', mode='r', shape=(BSM_SIZE, N_FEATURES))

df = pd.DataFrame(veremi, columns=[
    'dataset_id',
    'message_id',
    'bsm_rcv_time',
    'receiver',
    'sender',
    'attack_type',
    'snd_pos_x',
    'snd_pos_y',
    'snd_spd_x',
    'snd_spd_y',
    'snd_acl_x',
    'snd_acl_y',
    'snd_hed_x',
    'snd_hed_y',
    'rcv_pos_x',
    'rcv_pos_y',
    'rcv_spd_x',
    'rcv_spd_y',
    'rcv_acl_x',
    'rcv_acl_y',
    'rcv_hed_x',
    'rcv_hed_y',
])
# df.drop(df[df.attack_type.between(1, 9)].index, inplace=True) # drop faults (1 - 9)
df.sort_values(['dataset_id', 'sender', 'receiver', 'bsm_rcv_time'], inplace=True)
df.reset_index(inplace=True, drop=True)

# Calc delta_time based on the diff between the messages
def calc_delta(x):
    return pd.Series(np.insert(np.diff(x.bsm_rcv_time), 0, 0))

amount_per_iter = 1000000

for i in tqdm(range(math.ceil(len(df) / amount_per_iter))):
    curr_df = df[i * amount_per_iter:(i + 1) * amount_per_iter]
    grouped = curr_df[['dataset_id',
                       'sender',
                       'receiver',
                       'bsm_rcv_time']].groupby(['dataset_id',
                                            'sender',
                                            'receiver'])

    delta_time = grouped.apply(calc_delta)
    df.loc[curr_df.index, 'delta_time'] = delta_time.values

df.to_feather('out_veremi/veremi.feather')

  0%|          | 0/102 [00:00<?, ?it/s]

In [8]:
df = pd.read_feather('out_veremi/veremi.feather')

random_state = 42

# group by sender and receiver
# TODO: group only by sender
grouped = df.groupby(['dataset_id', 'sender', 'receiver', 'attack_type'])
interactions = grouped.size().rename('seq_len').reset_index()

train = interactions.sample(frac=0.8, random_state=random_state)
test = interactions.drop(train.index)

train_full_genuine = train[train.attack_type == 0]

sample_size_5_pctg = round(len(train_full_genuine) / 0.95 - len(train_full_genuine))
train_5_pctg_atk = train[train.attack_type != 0].sample(sample_size_5_pctg, random_state=random_state)
train_95_genuine = pd.concat([train_full_genuine, train_5_pctg_atk]).sample(frac=1, random_state=random_state) # concat and shuffle

sample_size_10_pctg = round(len(train_full_genuine) / 0.9 - len(train_full_genuine))
train_10_pctg_atk = train[train.attack_type != 0].sample(sample_size_10_pctg, random_state=random_state)
train_90_genuine = pd.concat([train_full_genuine, train_10_pctg_atk]).sample(frac=1, random_state=random_state) # concat and shuffle

del train_10_pctg_atk
del train_5_pctg_atk
del interactions
del grouped

print(f'Train: {len(train)}, Train 0%: {len(train_full_genuine)}, Train 5%: {len(train_95_genuine)}, Train 10%: {len(train_90_genuine)}, Test: {len(test)}')

Train: 3311449, Train 0%: 2321200, Train 5%: 2443368, Train 10%: 2579111, Test: 827862


In [9]:
cols = ['dataset_id', 'sender', 'receiver', 'attack_type']

test_df = df.join(test.set_index(cols), how='inner', on=cols)
test_df.reset_index(drop=True).to_feather('out_veremi/test.feather')
del test_df
del test

train_df = df.join(train.set_index(cols), how='inner', on=cols)
train_df.reset_index(drop=True).to_feather('out_veremi/train.feather')
del train_df
del train

train_full_genuine_df = df.join(train_full_genuine.set_index(cols), how='inner', on=cols)
train_full_genuine_df.reset_index(drop=True).to_feather('out_veremi/train_full_genuine.feather')
del train_full_genuine_df
del train_full_genuine

train_95_genuine_df = df.join(train_95_genuine.set_index(cols), how='inner', on=cols)
train_95_genuine_df.reset_index(drop=True).to_feather('out_veremi/train_95_genuine.feather')
del train_95_genuine_df
del train_95_genuine

train_90_genuine_df = df.join(train_90_genuine.set_index(cols), how='inner', on=cols)
train_90_genuine_df.reset_index(drop=True).to_feather('out_veremi/train_90_genuine.feather')
del train_90_genuine_df
del train_90_genuine